In [107]:
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import pandas as pd
import time
from datetime import datetime, timedelta
import time
from IPython.display import clear_output
from config import conn_host, conn_database, conn_user, conn_password
import mysql.connector
from dateutil import tz

In [108]:
months = dict(Jan=1,Feb=2,Mar=3,Apr=4,May=5,Jun=6,Jul=7,Aug=8,Sep=9,Oct=10,Nov=11,Dec=12)

# If you want to change the league, browse through the odds portal website to find the correct url parameters for you
country = "usa"
league = "mls"

In [109]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

In [110]:
execute_query("ALTER TABLE matches " +
              "ADD home_odds FLOAT NULL," +
              "ADD away_odds FLOAT NULL," +
              "ADD draw_odds FLOAT NULL;")

Execution failed on sql 'ALTER TABLE matches ADD home_odds FLOAT NULL,ADD away_odds FLOAT NULL,ADD draw_odds FLOAT NULL;': 1060 (42S21): Duplicate column name 'home_odds'


C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [111]:
teams = execute_query("SELECT * FROM teams")

In [112]:
print(teams)

        id            name
0       33  Manchester Utd
1       34       Newcastle
2       35     Bournemouth
3       36          Fulham
4       37    Huddersfield
..     ...             ...
156  12277        Ipatinga
157  13173      Chivas USA
158  13974      Santa Cruz
159  16489          Austin
160  18310       Charlotte

[161 rows x 2 columns]


In [113]:
def get_betting_odds(season):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/Sao_Paulo')
    base_url = f"https://www.oddsportal.com/soccer/{country}/{league}{season}/results/"
    option = Options()
#     option.headless = True
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)
    driver.get(base_url)
    time.sleep(5)
    
    try:
        button = driver.find_element("xpath", '/html/body/div[3]/div[3]/div/div[1]/div/div[2]/div/button[1]')
        button.click()
    except:
        print('A problem occoured while trying to close the cookies popup')
        
    button = driver.find_element("xpath", '/html/body/div[1]/div/div[2]/div[6]/div[1]/div/div[1]/div[2]/div[1]/div[6]/div/a[last()]')
    button.click()
    time.sleep(2)
    
    element = driver.find_element('id', "tournamentTable")
    htmlContent = element.get_attribute('outerHTML')
    page_soup = soup(htmlContent, "html.parser")
    active_page = int(''.join(page_soup.find('span', {"class": "active-page"}).findAll(text=True)))
    print('{}: Page {}'.format(season[1:], active_page))
    
    games = []
    
    while int(active_page) >= 1:
        dates = page_soup.findAll('tr', {"class": "center nob-border"})
        games_registered = 0
        
        for i in range(len(dates), 0, -1):
            try:
                if len(dates) == i:
                    date_games = dates[i-1].find_next_siblings('tr', {"class": "deactivate"})
                else:
                    date_games = dates[i-1].find_next_siblings('tr', {"class": "deactivate"})[:-1*games_registered]
                games_registered += len(date_games)
                date_info_splitted = dates[i-1].contents[0].text.split('-')
                if len(date_info_splitted) > 1:
                    continue
                date_text = date_info_splitted[0].strip()
                for g in date_games:
                    try:
                        unix_date = datetime(int(date_text.split(' ')[2]), 
                                                        months[date_text.split(' ')[1]], 
                                                        int(date_text.split(' ')[0]), 
                                                        hour=int(g.contents[0].text.strip().split(':')[0]), 
                                                        minute=int(g.contents[0].text.strip().split(':')[1]), 
                                                        second=0).replace(tzinfo=from_zone).astimezone(to_zone)
                        home_team_string = g.contents[1].text.split(' - ')[0].strip()
                        away_team_string = g.contents[1].text.split(' - ')[1].strip()
                        home_team = teams.loc[teams['name'] == home_team_string].iloc[0]
                        away_team = teams.loc[teams['name'] == away_team_string].iloc[0]
                        games_parsed = [unix_date,
                                          home_team['id'], # Home Team Id
                                          away_team['id'], # Away Team Id
                                          home_team['name'], # Team A Name
                                          away_team['name'], # Team B Name
                                          g.contents[2].text.split(':')[0].strip(), # Team A Score
                                          g.contents[2].text.split(':')[1].replace('OT', '').strip(), # Team B Score
                                          g.contents[3].text,  # Team A Odds
                                          g.contents[4].text,  # Draw Odds
                                          g.contents[5].text]  # Team B Odds
                        games.append(games_parsed)
                    except:
                        continue
                # print('{} Games appended'.format(len(games_parsed)))
            except Exception as e:
                print(e)
                continue
        
        btn_next_page = driver.find_element("xpath", '/html/body/div[1]/div/div[2]/div[6]/div[1]/div/div[1]/div[2]/div[1]/div[6]/div/a[2]')
        btn_next_page.click()
        time.sleep(4)
        
        element = driver.find_element('id', "tournamentTable")
        htmlContent = element.get_attribute('outerHTML')
        page_soup = soup(htmlContent, "html.parser")
        active_page -= 1
        clear_output(wait=True)
        print('{}: Page {}'.format(season[1:], active_page))
    
    driver.quit()
    return games
    

In [114]:
season = 2014
games = []
while season < 2022:
    games.extend(get_betting_odds(f'-{season}')) # Brazil/Same year leagues
#     games.extend(get_betting_odds(f'-{season}-{season+1}')) # European/Two year leagues leagues
    season += 1

games.extend(get_betting_odds('')) # For the current season

odds_df = pd.DataFrame(games, columns=['GAME_DATE', 'TEAM_A_ID', 'TEAM_B_ID', 'TEAM_A', 'TEAM_B', 'TEAM_A_SCORE', 'TEAM_B_SCORE', 'TEAM_A_ODDS', 'DRAW_ODDS', 'TEAM_B_ODDS'])
# odds_df.to_csv('../data/odds.csv')
print(len(games))

: Page 0
2880


In [115]:
odds_df.head()

,GAME_DATE,TEAM_A_ID,TEAM_B_ID,TEAM_A,TEAM_B,TEAM_A_SCORE,TEAM_B_SCORE,TEAM_A_ODDS,DRAW_ODDS,TEAM_B_ODDS
0,2014-03-08 21:30:00-03:00,1603,1602,Vancouver Whitecaps,New York Red Bulls,4,1,2.29,3.26,3.12
1,2014-03-08 21:00:00-03:00,1615,1613,DC United,Columbus Crew,0,3,2.31,3.24,3.16
2,2014-03-09 17:00:00-03:00,1595,1611,Seattle Sounders,Sporting Kansas City,1,0,2.15,3.23,3.43
3,2014-03-09 16:00:00-03:00,13173,1607,Chivas USA,Chicago Fire,3,2,3.10,3.32,2.27
4,2014-03-09 00:30:00-03:00,1605,1606,Los Angeles Galaxy,Real Salt Lake,0,1,1.70,3.60,5.10


In [116]:
game_errors = []
for index, row in odds_df.iterrows():
    try:
        date_minus_one_day = row['GAME_DATE'] - timedelta(days=1)
        date_plus_one_day = row['GAME_DATE'] + timedelta(days=1)
        game_id = execute_query(f"SELECT id from matches WHERE (DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{row['GAME_DATE']}','%d/%m/%Y') OR DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{date_minus_one_day}','%d/%m/%Y') OR DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{date_plus_one_day}','%d/%m/%Y')) AND home_id = {row['TEAM_A_ID']} AND away_id = {row['TEAM_B_ID']}").iloc[0]['id']
        update_query = f"UPDATE matches SET home_odds = {row['TEAM_A_ODDS']}, away_odds = {row['TEAM_B_ODDS']}, draw_odds = {row['DRAW_ODDS']} WHERE id = {game_id}"
        execute_query(update_query, False)
    except Exception as e:
        new_item = dict()
        new_item['error'] = e
        new_item['match'] = row
        game_errors.append(new_item)
    clear_output(wait=True)
    print(f"{index}/{len(odds_df.index)}")

2879/2880


In [117]:
if len(game_errors) == 0:
    print('There were no errors while reading games from the Odds Portal website.')
else:
    print('The following games had an error while trying to read games from the Odds Portal website:')
    for item in game_errors:
        print(f"\n{item['match']['GAME_DATE']}: {item['match']['TEAM_A']}({item['match']['TEAM_A_ID']}) {item['match']['TEAM_A_SCORE']} x {item['match']['TEAM_B_SCORE']} {item['match']['TEAM_B']}({item['match']['TEAM_B_ID']})")
        print(f"ERROR: {item['error']}")

The following games had an error while trying to read games from the Odds Portal website:

2014-03-08 21:30:00-03:00: Vancouver Whitecaps(1603) 4 x 1 New York Red Bulls(1602)
ERROR: single positional indexer is out-of-bounds

2014-03-16 20:00:00-03:00: Chivas USA(13173) 1 x 1 Vancouver Whitecaps(1603)
ERROR: single positional indexer is out-of-bounds

2014-03-23 15:00:00-03:00: New England Revolution(1609) 0 x 0 Vancouver Whitecaps(1603)
ERROR: single positional indexer is out-of-bounds

2014-03-30 20:00:00-03:00: Vancouver Whitecaps(1603) 2 x 1 Houston Dynamo(1600)
ERROR: single positional indexer is out-of-bounds

2014-04-05 19:30:00-03:00: Vancouver Whitecaps(1603) 1 x 2 Colorado Rapids(1610)
ERROR: single positional indexer is out-of-bounds

2014-04-12 23:30:00-03:00: Los Angeles Galaxy(1605) 1 x 0 Vancouver Whitecaps(1603)
ERROR: single positional indexer is out-of-bounds

2014-04-19 20:00:00-03:00: Vancouver Whitecaps(1603) 2 x 2 Los Angeles Galaxy(1605)
ERROR: single positional 

ERROR: single positional indexer is out-of-bounds


In [118]:
fixtures_df = execute_query(f"SELECT m.id, m.date, m.season, l.name AS league, ht.name as home_team, at.name as away_team, m.home_score, m.away_score, m.home_odds, m.away_odds, m.draw_odds FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) WHERE m.season = 2021 ORDER BY m.date ASC")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [119]:
fixtures_df.head()

,id,date,season,league,home_team,away_team,home_score,away_score,home_odds,away_odds,draw_odds
0,684127,2021-04-16 21:00:00,2021,Major League Soccer (USA),Houston Dynamo,San Jose Earthquakes,2,1,1.82,4.00,3.95
1,684128,2021-04-16 22:30:00,2021,Major League Soccer (USA),Seattle Sounders,Minnesota United,4,0,1.78,4.21,3.95
2,684130,2021-04-17 15:00:00,2021,Major League Soccer (USA),CF Montreal,Toronto FC,4,2,2.95,2.32,3.57
3,684131,2021-04-17 16:00:00,2021,Major League Soccer (USA),Orlando City SC,Atlanta Utd,0,0,NaN,NaN,NaN
4,684129,2021-04-17 21:00:00,2021,Major League Soccer (USA),New York Red Bulls,Sporting Kansas City,1,2,2.15,3.27,3.59
